In [4]:
!pip install langchain-text-splitters
!pip install sentence-transformers
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.7 MB/s eta 0:00:00


# load Văn bản hợp nhất-03-VBHN-BGTVT

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
file_path = '/content/Văn bản hợp nhất-03-VBHN-BGTVT.docx.txt'
with open(file_path, 'r') as file:
            content = file.read()

# split content to chunk

In [3]:
from transformers import AutoTokenizer
from pyvi.ViTokenizer import tokenize as pyvi_tokenizer

tokenizer = AutoTokenizer.from_pretrained('dangvantuan/vietnamese-embedding')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
    length_function=lambda x: len(tokenizer(pyvi_tokenizer(x))["input_ids"]),
    is_separator_regex=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
chunks_ = text_splitter.create_documents([content])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

Token indices sequence length is longer than the specified maximum sequence length for this model (64612 > 512). Running this sequence through the model will result in indexing errors


The text has been broken down in 418 chunks.


In [5]:
max([len(chunk.split()) for chunk in chunks]), max([len(tokenizer(pyvi_tokenizer(chunk))["input_ids"]) for chunk in chunks])

(311, 249)

# convert chunk to vector
I use dangvantuan/vietnamese-embedding model for embedding model.

In [7]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding').cuda() #('thenlper/gte-large').cuda()
embeddings = embedding_model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.9351]])


In [8]:
embeddings.shape

(2, 768)

In [9]:
import torch
import numpy as np

embeddings_ = []
batch_size = 8

for _id in range(0, len(chunks)//batch_size + int(len(chunks) % batch_size != 0)):
    start_id = _id * batch_size
    end_id = min(len(chunks), start_id + batch_size)
    print(start_id, end_id)
    batch_chunks = [pyvi_tokenizer(chunk) for chunk in chunks[start_id:end_id]]
    with torch.no_grad():
        embeddings = embedding_model.encode(batch_chunks)
    embeddings_.append(embeddings)

embeddings = torch.tensor(np.concatenate(embeddings_, 0))
embeddings.shape

0 8
8 16
16 24
24 32
32 40
40 48
48 56
56 64
64 72
72 80
80 88
88 96
96 104
104 112
112 120
120 128
128 136
136 144
144 152
152 160
160 168
168 176
176 184
184 192
192 200
200 208
208 216
216 224
224 232
232 240
240 248
248 256
256 264
264 272
272 280
280 288
288 296
296 304
304 312
312 320
320 328
328 336
336 344
344 352
352 360
360 368
368 376
376 384
384 392
392 400
400 408
408 416
416 418


torch.Size([418, 768])

# load LLM model
I use Qwen 2.5 7B Instruct

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# convert question to embedding

In [44]:
#question = 'xe máy vi phạm nồng độ cồn bị phạt bao nhiêu tiền?'
question = 'ô tô vượt đèn đỏ bị phạt bao nhiêu tiền?'
query_embedding = embedding_model.encode([pyvi_tokenizer(question)])
query_embedding = torch.tensor(query_embedding)
query_embedding.shape

torch.Size([1, 768])

# calulate top 20 similar chunk by cosine

In [45]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
scores = cos(query_embedding.cuda(), embeddings.cuda())

In [46]:
indices = scores.topk(20).indices.cpu().long().numpy()
indices

array([ 31,  34,  45, 105, 110, 101, 103,  98, 109,  33, 112,  27, 200,
       114,  47,  46, 182, 173,  62, 142])

In [47]:
import numpy as np
chunks_ = np.array(chunks)
chunks_[indices]

array(['h)[20] (được bãi bỏ)\ni) Điều khiển xe chạy quá tốc độ quy định từ 10 km/h đến 20 km/h.\n6. Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:\na) Điều khiển xe chạy quá tốc độ quy định trên 20 km/h đến 35 km/h;\nb)[21] Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ;\nc) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.\n7. Phạt tiền từ 10.000.000 đồng đến 12.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:',
       '9. Phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng đối với hành vi vi phạm quy định tại điểm b khoản 7 Điều này mà không chấp hành hiệu lệnh dừng xe của người thi hành công vụ hoặc gây tai nạn giao thông.\n10. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với người điều khiển xe thực 

# create question prompt

In [48]:
Contexts = "\n".join(chunks_[indices].tolist())
print(Contexts)

h)[20] (được bãi bỏ)
i) Điều khiển xe chạy quá tốc độ quy định từ 10 km/h đến 20 km/h.
6. Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
a) Điều khiển xe chạy quá tốc độ quy định trên 20 km/h đến 35 km/h;
b)[21] Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ;
c) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
7. Phạt tiền từ 10.000.000 đồng đến 12.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
9. Phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng đối với hành vi vi phạm quy định tại điểm b khoản 7 Điều này mà không chấp hành hiệu lệnh dừng xe của người thi hành công vụ hoặc gây tai nạn giao thông.
10. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành v

In [49]:
question_prompt = f'''Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{Contexts}

Now here is the question you need to answer.

Question: {question}'''

In [50]:
print(question_prompt)

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
h)[20] (được bãi bỏ)
i) Điều khiển xe chạy quá tốc độ quy định từ 10 km/h đến 20 km/h.
6. Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
a) Điều khiển xe chạy quá tốc độ quy định trên 20 km/h đến 35 km/h;
b)[21] Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ;
c) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
7. Phạt tiền từ 10.000.000 đồng đến 12.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm 

# Ask Qwen2 7B
Sorry because I don't have gpu to run it :(

In [18]:
device = "cuda" # the device to load the model onto

messages = [
    {"role": "system", "content": "You are a helpful assistant. Please answer question more detail base on context!"},
    {"role": "user", "content": question_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.26 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.92 GiB is free. Process 77212 has 12.83 GiB memory in use. Of the allocated memory 12.38 GiB is allocated by PyTorch, and 331.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Ask Qwen2.5 72B
Because I can't run it on Colab Free tier, I use Together AI to test Qwen2.5 72B

In [19]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 3.5 MB/s eta 0:00:00


In [51]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Dựa trên thông tin được cung cấp, hành vi điều khiển xe ô tô không chấp hành hiệu lệnh của đèn tín hiệu giao thông (vượt đèn đỏ) bị phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng (theo khoản 4, điểm e).


In [52]:
import json

model = "Qwen2.5-72B-Instruct-Turbo"

with open(f"{question}_{model}.txt", 'w', encoding='utf-8') as f:
    f.write(f"#question_prompt:\n\n\n {question_prompt} \n\n\n#answer:\n\n\n {response.choices[0].message.content}")

# Ask Llama 3.1 405B
Because I can't run it on Colab Free tier, I use Together AI to test Llama 3.1 405B

In [53]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Không có thông tin cụ thể về mức phạt cho hành vi "ô tô vượt đèn đỏ" trong văn bản đã cung cấp. Tuy nhiên, có thông tin về mức phạt cho hành vi "không chấp hành hiệu lệnh của đèn tín hiệu giao thông" tại điểm e khoản 4 là từ 800.000 đồng đến 1.000.000 đồng.


In [54]:
import json

model = "Meta-Llama-3.1-405B-Instruct-Turbo"

with open(f"{question}_{model}.txt", 'w', encoding='utf-8') as f:
    f.write(f"#question_prompt:\n\n\n {question_prompt} \n\n\n#answer:\n\n\n {response.choices[0].message.content}")

# The result base on LLM model and embedding model.

## The relevant chunk don't show enough contexts than LLm can't answer correctly!

## Chunks only split by token len, they don't have enough context about Article or Clause. Than the context maybe unclear for LLM.

## We need to rerank relevant chunk after get top 20 similar chunk to improve result

## I just use basic chunking strategy, maybe advanced chunking can help improve RAG QA system (late chunking, semantic chunking) or use longer context embedding model

## Spelling errors in the text may occur when downloading as a .txt file. Perhaps we need to clean the data for better results.